In [4]:
import requests
from bs4 import BeautifulSoup

url = 'https://fruggies.com/'
response = requests.get(url)
content = response.content

soup = BeautifulSoup(content, 'html.parser')
data = []

for item in soup.find_all('div', {'class': 'product'}):
    title = item.find('h3', {'class': 'product-name'}).text.strip()
    price = item.find('div', {'class': 'price'}).text.strip()
    image = item.find('img', {'class': 'attachment-shop_catalog'})['src']
    description = item.find('div', {'class': 'product-description'}).text.strip()

    data.append({
        'title': title,
        'price': price,
        'image': image,
        'description': description
    })

print(data)

[]


In [2]:
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker


engine = create_engine('sqlite:///fruggies.db', echo=True)

Session = sessionmaker(bind=engine)
session = Session()

Base = declarative_base()
class Product(Base):
    __tablename__ = 'products'

    id = Column(Integer, primary_key=True)
    title = Column(String)
    price = Column(String)
    image = Column(String)
    description = Column(String)

Base.metadata.create_all(engine)

for item in data:
    product = Product(title=item['title'], price=item['price'], image=item['image'], description=item['description'])
    session.add(product)

session.commit()

2023-03-18 12:14:50,913 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-18 12:14:50,914 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("products")
2023-03-18 12:14:50,915 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-18 12:14:50,919 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("products")
2023-03-18 12:14:50,920 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-18 12:14:50,922 INFO sqlalchemy.engine.Engine 
CREATE TABLE products (
	id INTEGER NOT NULL, 
	title VARCHAR, 
	price VARCHAR, 
	image VARCHAR, 
	description VARCHAR, 
	PRIMARY KEY (id)
)


2023-03-18 12:14:50,923 INFO sqlalchemy.engine.Engine [no key 0.00100s] ()
2023-03-18 12:14:50,931 INFO sqlalchemy.engine.Engine COMMIT


In [4]:
pip install streamlit

Defaulting to user installation because normal site-packages is not writeable
  Using cached streamlit-1.20.0-py2.py3-none-any.whl (9.6 MB)
  Using cached Pympler-1.0.1-py3-none-any.whl (164 kB)
     --------------------------------------- 20.6/20.6 MB 40.9 MB/s eta 0:00:00
  Using cached blinker-1.5-py2.py3-none-any.whl (12 kB)
  Using cached altair-4.2.2-py3-none-any.whl (813 kB)
  Using cached validators-0.20.0.tar.gz (30 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached pydeck-0.8.0-py2.py3-none-any.whl (4.7 MB)
     ------------------------------------- 904.2/904.2 kB 28.8 MB/s eta 0:00:00
  Using cached GitPython-3.1.31-py3-none-any.whl (184 kB)
  Using cached rich-13.3.2-py3-none-any.whl (238 kB)
  Using cached tzlocal-4.2-py3-none-any.whl (19 kB)
  Using cached cachetools-5.3.0-py3-none-any.whl (9.3 kB)
  Using cached semver-2.13.0-py2.py3-none-any.whl (12 kB)
  Using cached gitdb-4.0.10-py3-none-any.whl (62

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.2.2 requires pyqt5<5.13, which is not installed.
spyder 5.2.2 requires pyqtwebengine<5.13, which is not installed.


In [10]:
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.automap import automap_base

engine = create_engine('sqlite:///fruggies.db')

Session = sessionmaker(bind=engine)
session = Session()

Base = automap_base()
Base.prepare(engine, reflect=True)
Product = Base.classes.products

query = session.query(Product)
results = query.all()
df = pd.DataFrame([(r.title, r.price, r.description) for r in results], columns=['Title', 'Price', 'Description'])

st.dataframe(df)

plt.bar(df['Title'], df['Price'])
plt.xticks(rotation=90)
st.pyplot()

DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

In [14]:
pip install scraper

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install database

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement database (from versions: none)
ERROR: No matching distribution found for database


In [16]:
import unittest
from scraper import scrape_data
from database import insert_data
from dashboard import display_data

class TestScraper(unittest.TestCase):

    def test_scrape_data(self):
        data = scrape_data('https://fruggies.com/')
        self.assertGreater(len(data), 0)

class TestDatabase(unittest.TestCase):

    def test_insert_data(self):
        engine = create_engine('sqlite:///fruggies.db')
        Base.metadata.create_all(engine)
        Session = sessionmaker(bind=engine)
        session = Session()
        insert_data(session, data)
        product_count = session.query(Product).count()
        self.assertEqual(len(data), product_count)

class TestDashboard(unittest.TestCase):

    def test_display_data(self):
        engine = create_engine('sqlite:///fruggies.db')
        Base = automap_base()
        Base.prepare(engine, reflect=True)
        Product = Base.classes.products
        Session = sessionmaker(bind=engine)
        session = Session()
        query = session.query(Product)
        results = query.all()
        df = pd.DataFrame([(r.title, r.price, r.description) for r in results], columns=['Title', 'Price', 'Description'])
        self.assertGreater(len(df), 0)

if __name__ == '__main__':
    unittest.main()

ImportError: cannot import name 'scrape_data' from 'scraper' (C:\Users\Nadia\AppData\Roaming\Python\Python39\site-packages\scraper\__init__.py)